In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler
driver=webdriver.Chrome(executable_path=r'chromedriver.exe')
driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=RELIANCE')
html=driver.page_source
driver.close()
sel_soup=BeautifulSoup(html,'html.parser')
cht2=len(sel_soup.find_all(class_='grybg'))
price=[]
time=[]
final_call={}
final_put={}
call_ltp={}
put_ltp={}
for i in range(0,cht2):
    final=float(sel_soup.find_all(class_='grybg')[i].find_all('b')[0].get_text())
    price.append(final)
final_price=price[9:20]
for price1 in final_price:
    final_call[price1]=[]
    final_put[price1]=[]
price_ltp=price[9:20]
for price1 in price_ltp:
    call_ltp[price1]=[]
    put_ltp[price1]=[]  

In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler
sched = BackgroundScheduler()
@sched.scheduled_job('interval', seconds=180)
def timed_job():
    driver=webdriver.Chrome(executable_path=r'chromedriver.exe')
    driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol=RELIANCE')
    html=driver.page_source
    driver.quit()
    sel_soup=BeautifulSoup(html,'html.parser')
    global time
    global final_call
    global final_put
    global call_ltp
    global put_ltp
    time.append(datetime.now())
    for data in range(14,25):
        strike=float(sel_soup.find_all('tr')[data].find_all('td')[11].text.strip())
        call=''
        put=''
        for value in sel_soup.find_all('tr')[data].find_all('td')[1].text.strip().split(','):
            call=call+value
        for value in sel_soup.find_all('tr')[data].find_all('td')[-2].text.strip().split(','):
            put=put+value
        call=int(call)
        put=int(put)
        final_call[strike].append(call)
        final_put[strike].append(put)
    for data in range(14,25):
        strike=float(sel_soup.find_all('tr')[data].find_all('td')[11].text.strip())
        call_price=float(sel_soup.find_all('tr')[data].find_all('td')[5].text.strip())
        put_price=float(sel_soup.find_all('tr')[data].find_all('td')[-6].text.strip())
        call_ltp[strike].append(call_price)
        put_ltp[strike].append(put_price)      
sched.start()    

In [ ]:
final_price

In [ ]:
#Individual plot LTP for put and call
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set_style('whitegrid')
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
put=[]
call=[]
option_time=time[3:]
value=1950.0
call_length=len(call_ltp[value])
put_length=len(put_ltp[value])
for i in range(1,call_length):
    y=((call_ltp[value][i]-call_ltp[value][0])/call_ltp[value][0])
    w=(y*100)
    call.append(w)
for i in range(1,put_length):
    y=((put_ltp[value][i]-put_ltp[value][0])/put_ltp[value][0])
    w=(y*100)
    put.append(w)
df = pd.DataFrame(list(zip(option_time,call,put)),columns =['Time', 'Call','Put'])
df.set_index(['Time'],inplace =True)
df.iplot()

In [ ]:
#Individual
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set_style('whitegrid')
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
put=[]
call=[]
option_time=time[3:]
value=1950.0
call_length=len(final_call[value])
put_length=len(final_put[value])
for i in range(1,call_length):
    y=((final_call[value][i]-final_call[value][0])/final_call[value][0])
    w=(y*100)
    call.append(w)
for i in range(1,put_length):
    y=((final_put[value][i]-final_put[value][0])/final_put[value][0])
    w=(y*100)
    put.append(w)
df = pd.DataFrame(list(zip(option_time,call,put)),columns =['Time', 'Call','Put'])
df.set_index(['Time'],inplace =True)
df.iplot()

In [ ]:
#Overall
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set_style('whitegrid')
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
put1=[]
call1=[]
price_value=[]
for key in final_call.keys():
    price_value.append(key)
    put1.append(((final_put[key][-1]-final_put[key][0])/final_put[key][0])*100)
    call1.append(((final_call[key][-1]-final_call[key][0])/final_call[key][0])*100)
frame = pd.DataFrame(list(zip(price_value,call1,put1)),columns =['Price', 'Call','Put'])
frame.set_index(['Price'],inplace =True)
frame.iplot()

In [ ]:
sched.shutdown()